In [ ]:
import pandas as pd
import sqlalchemy

In [9]:
def sample_random_warehouse(fraction = 0.1):
    sql_statement = text(f"SELECT warehouseId FROM snowset TABLESAMPLE SYSTEM ({fraction})")
    result_df = None
    with engine.connect() as conn:
        result_df = pd.DataFrame(conn.execute(sql_query).fetchall(), columns=['warehouseId'])

    return result_df


In [6]:
sql_query = sqlalchemy.text("SELECT warehouseId FROM snowset TABLESAMPLE SYSTEM (0.1)")
result_df = None
with engine.connect() as conn:
    result_df = pd.DataFrame(conn.execute(sql_query).fetchall(), columns=['warehouseId'])

result_df

,warehouseId
0,6447098146458688922
1,6447098146458688922
2,6447098146458688922
3,6447098146458688922
4,6447098146458688922
...,...
375,6025240988369907361
376,6314450707633251764
377,483532010532309028
378,483532010532309028


In [ ]:
sql_statement = sqlalchemy.text("""
   SELECT s.warehouseId as warehouseId,
   sum(systemCpuTime) + sum(userCpuTime) AS cpuMicros,
   sum(persistentReadBytesS3)            AS scanS3,
   sum(persistentReadBytesCache)         AS scanCache,
   sum(intDataReadBytesLocalSSD)         AS spoolSSD,
   sum(intDataReadBytesS3)               AS spoolS3,
   avg(warehousesize)                    AS warehouse_size
  FROM snowset s
  JOIN (SELECT warehouseId FROM snowset TABLESAMPLE SYSTEM (0.01)) w ON w.warehouseId = s.warehouseId
  -- WHERE s.warehouseSize = 1
  GROUP BY s.warehouseId
""")

result_df = None
with engine.connect() as conn:
    result_df = pd.DataFrame(conn.execute(sql_statement).fetchall(), columns=[
        'warehouseId',
        'cpuMicros',
        'scanS3',
        'scanCache',
        'spoolSSD',
        'spoolS3',
        'warehouse_size'
    ])

result_df

KeyboardInterrupt: 